This notebook generates sample data that can be used to illustrate the Cloud ML Ecosystem workflow for using AEP Data for machine learning use cases with external ML tools and platforms. We will generate sample data with the following steps:

- [Setup](#setup)
- [1. Create Experience Event schema and dataset](#1-create-experience-event-schema-and-dataset)
- [2. Statistical simulation of Experience Events](#2-statistical-simulation-of-experience-events)
- [3. Ingest sythetic data into AEP dataset](#3-ingest-sythetic-data-into-aep-dataset)



# Setup

Before we run anything, make sure to install the following required libraries for this notebook. They are all publicly available libraries and the latest version should work fine.

In [ ]:
%pip install mmh3
%pip install rstr
%pip install aepp
%pip install pygresql

This notebook requires some configuration data to properly authenticate to your Adobe Experience Platform instance. You should be able to find all the values required above by following the Setup section of the **README**.

The next cell will be looking for your configuration file under your **ADOBE_HOME** path to fetch the values used throughout this notebook. See more details in the Setup section of the **README** to understand how to create your configuration file.

In [ ]:
import os
from configparser import ConfigParser
import aepp

os.environ["ADOBE_HOME"] = os.path.dirname(os.getcwd())

if "ADOBE_HOME" not in os.environ:
    raise Exception("ADOBE_HOME environment variable needs to be set.")

config = ConfigParser()
config_file = "cmle_gov_config.ini"
config_path = os.path.join(os.environ["ADOBE_HOME"], "conf", config_file)

if not os.path.exists(config_path):
    raise Exception(f"Looking for configuration under {config_path} but config not found, please verify path")

config.read(config_path)

aepp.configure(
  org_id=config.get("Platform", "ims_org_id"),
  tech_id=config.get("Authentication", "tech_acct_id"), 
  secret=config.get("Authentication", "client_secret"),
  scopes=config.get("Authentication", "scopes"),
  client_id=config.get("Authentication", "client_id"),
  environment=config.get("Platform", "environment"),
  sandbox=config.get("Platform", "sandbox_name")
)

To ensure uniqueness of resources created as part of this notebook, we are using your local username to include in each of the resource titles to avoid conflicts.

In [ ]:
import re
username = os.getlogin()
unique_id = s = re.sub("[^0-9a-zA-Z]+", "_", username)

Some utility functions that will be used throughout this notebook:

In [ ]:
def get_ui_link(tenant_id, resource_type, resource_id):
    environment = config.get("Platform", "environment")
    sandbox_name = config.get("Platform", "sandbox_name")
    if environment == "prod":
        prefix = f"https://experience.adobe.com"
    else:
        prefix = f"https://experience-{environment}.adobe.com"
    return f"{prefix}/#/@{tenant_id}/sname:{sandbox_name}/platform/{resource_type}/{resource_id}"

# 1. Create Experience Event schema and dataset

We will now create the schema to support our synthetic data. We need a few fields which will be included in the synthetic event data:
- Direct Marketing information
- Web details

## 1.1 Create connection to XDM Schema Registry

In [ ]:
from aepp import schema
schema_conn = schema.Schema()
schema_conn.sandbox
tenant_id = schema_conn.getTenantId()
tenant_id

## Create User ID field group for Experience Event Schema

We need to create a custom field group with a "userId" field to go in the experience event schema. Other schema fields will come from standard field field groups we will include when creating the schema.

First we'll define some utility functions to gracefully handle cases where the User ID field group has already been created:

In [ ]:
def getFieldGroupbyTitle(schema_conn: schema.Schema, title: str):
    fieldgroups = schema_conn.getFieldGroups()
    match = list(filter(lambda d: d['title'] == title, fieldgroups))
    if len(match) == 1:
        return match[0]
    else:
        return None

In [ ]:
def createFieldGroupifnotExists(schema_conn: schema.Schema, title: str, data: dict):
    existing = getFieldGroupbyTitle(schema_conn, title)
    if existing:
        print(f"'{title}' already exists, retrieving existing field group")
        return existing
    else:
        return schema_conn.createFieldGroup(data)

Create the User ID field group (or retrieve the field group ID if it already exists):

In [ ]:
fieldgroup_title = f"[CMLE Synthetic Data] Exp Event User ID (created by {username})"
fieldgroup_data = {
  "type": "object",
	"title": fieldgroup_title,
	"description": "This field group is used to identify the user to whom an experience event belongs.",
	"allOf": [{
		"$ref": "#/definitions/customFields"
	}],
	"meta:containerId": "tenant",
	"meta:resourceType": "mixins",
	"meta:xdmType": "object",
	"definitions": {
      "customFields": {
        "type": "object",
        "properties": {
          f"_{tenant_id}": {
            "type": "object",
            "properties": {
              "userId": {
                "title": "User ID",
                "description": "This refers to the user having a propensity towards an outcome.",
                "type": "string"
              }
            }
          }
        }
      }
	},
	"meta:intendedToExtend": ["https://ns.adobe.com/xdm/context/experienceevent"]
}

In [ ]:
fieldgroup_res = createFieldGroupifnotExists(schema_conn, fieldgroup_title, fieldgroup_data)
fieldgroup_id = fieldgroup_res['$id']
print(f"User ID field group ID: {fieldgroup_id}")

# Get link to field group in AEP UI
import urllib.parse
fieldgroup_link = get_ui_link(tenant_id, "schema/mixin/browse", urllib.parse.quote(fieldgroup_id, safe="a"))
print(f"View field group in UI: {fieldgroup_link}")

## 1.2 Compose Experience Event schema

Now we'll create the experience event schema from our custom field group and the following standard field groups:
- Direct Marketing Details
- Web Details

First we'll define some utility functions to gracefully handle cases where the Experience Event schema has already been created:

In [ ]:
def getSchemabyTitle(schema_conn: schema.Schema, title: str):
    schemas = schema_conn.getSchemas()
    # Handle case where no schemas have been created
    if 'results' in schemas: 
        return None
    # Filter schemas list for matching title
    match = list(filter(lambda d: d['title'] == title, schemas))
    # XDM schema titles must be unique, so 'match' will have exactly 1 element if a schema
    # with the same title already exists
    if len(match) == 1:
        return match[0]
    else:
        return None

In [ ]:
def createEESchemaifnotExists(schema_conn: schema.Schema, title: str, fieldGroups: list[str], description: str = ""):
    existing = getSchemabyTitle(schema_conn, title)
    if existing:
        print(f"'{title}' already exists, retrieving existing schema")
        return existing
    else:
        return schema_conn.createExperienceEventSchema(
            name=title,
            fieldGroupIds=fieldGroups,
            description=description
        )

Create the Experience Event schema (or retrieve the ID and Alt ID if the schema already exists):

In [ ]:
schema_ee_title = f"[CMLE Synthetic Data] Experience Event schema (created by {username})"
schema_ee_fgs = [
    fieldgroup_id,
    "https://ns.adobe.com/xdm/context/experienceevent-directmarketing",
    "https://ns.adobe.com/xdm/context/experienceevent-web"
]
schema_ee_desc = "Profile Schema generated by CMLE for synthetic events"

In [ ]:
schema_ee_res = createEESchemaifnotExists(
    schema_conn=schema_conn,
    title=schema_ee_title,
    fieldGroups=schema_ee_fgs,
    description=schema_ee_desc
)
schema_ee_id = schema_ee_res['$id']
schema_ee_altId = schema_ee_res["meta:altId"]
print(f"EE Schema ID: {schema_ee_id}")
print(f"EE Schema Alt ID: {schema_ee_altId}")

schema_ee_link = get_ui_link(tenant_id, "schema/mixin/browse", urllib.parse.quote(schema_ee_id, safe="a"))
print(f"View EE schema in UI: {schema_ee_link}")

Set "userId" as the primary ID for the schema with ECID as the namespace

In [ ]:
identity_type = "ECID"
identity_desc_data = {
    "@type": "xdm:descriptorIdentity",
    "xdm:sourceSchema": schema_ee_id,
    "xdm:sourceVersion": 1,
    "xdm:sourceProperty": f"/_{tenant_id}/userId",
    "xdm:namespace": identity_type,
    "xdm:property": "xdm:id",
    "xdm:isPrimary": True
  }
identity_dsc_ee_res = schema_conn.createDescriptor(
    descriptorObj = identity_desc_data
)
identity_dsc_ee_res

Enable EE schema for Profile

In [ ]:
enable_ee_res = schema_conn.enableSchemaForRealTime(schema_ee_altId)
enable_ee_res

## 1.3 Create Experience Event dataset

First, create a connection to the Catalog API

In [ ]:
from aepp import catalog
cat_conn = catalog.Catalog()

Define some utility functions to gracefully handle cases where the Experience Event dataset has alrerady been created:

In [ ]:
def getDatasetbyName(cat_conn: catalog.Catalog, name: str):
    datasets = cat_conn.getDataSets()
    match = {k:v for k, v in datasets.items() if v['name'] == name}
    if match:
        return list(match.keys())[0]
    else:
        return None

In [ ]:
def createDatasetifNotExists(cat_conn: catalog.Catalog, name: str, schemaId: str):
    existing = getDatasetbyName(cat_conn=cat_conn, name=name)
    if existing:
        return existing
    else:
        dataset = cat_conn.createDataSets(name=name, schemaId=schemaId)
        return dataset[0].split("/")[-1]

Create the Experience Event dataset

In [ ]:
dataset_ee_name = f"[CMLE Synthetic Data] Experience Event dataset (created by {username})"
dataset_ee_id = createDatasetifNotExists(cat_conn=cat_conn, name=dataset_ee_name, schemaId=schema_ee_id)
print(f"EE Dataset ID: {dataset_ee_id}")

dataset_ee_link = get_ui_link(tenant_id, "dataset/browse", dataset_ee_id)
print(f"View EE Dataset in UI: {dataset_ee_link}")


Enable dataset for Profile
<div class="alert alert-block alert-warning">
<b>Note:</b> After you do this step please go in the UI and click on the link above, if the profile toggle is not enabled please manually toggle the profile on
</div>

In [ ]:
cat_conn.enableDatasetProfile(dataset_ee_id)

# 2. Statistical simulation of Experience Events

We will set up a statistical simulation to generate Experience event data that can be used illustrate the end-to-end flow of creating a propensity model to predict subscriptions to a brand's paid service.

We will use the standard `web.formFilledOut` event type to represent the subscription conversions that the brand wants to predict, and generate similulated sequences of various types of experience events along with the target subscription conversions that will be used to train a propensity model.

## 2.1 Event types and their contribution to propensity



In [115]:
import random, string
import uuid
from datetime import timedelta
import mmh3
from random import randrange

Define some events and dependencies between the events

In [116]:
advertising_events = {
 
    #eventType          : (weeklyAverageOccurrence, propensityDelta, [(field_to_replace, value)], timeInHoursFromDependent)
    "advertising.clicks": (0.01,                    0.002,            [("advertising/clicks/value", 1.0)], 0.5) , 
    "advertising.impressions": (0.1, 0.001, [("advertising/impressions/value", 1.0)], 0),

    "web.webpagedetails.pageViews": (0.1, 0.005, [("web/webPageDetails/pageViews/value", 1.0)], 0.1),
    "web.webinteraction.linkClicks": (0.05, 0.005, [("web/webInteraction/linkClicks/value", 1.0)], 0.1),
   
    
    "commerce.productViews": (0.05, 0.005, [("commerce/productViews/value", 1.0)], 0.2),
    "commerce.purchases": (0.01, 0.1, [("commerce/purchases/value", 1.0)], 1),
    
    
    "decisioning.propositionDisplay": (0.05, 0.005, [("_experience/decisioning/propositionEventType/display", 1)], 0.1),
    "decisioning.propositionInteract": (0.01, 0.1, [("_experience/decisioning/propositionEventType/interact", 1)], 0.05),
    "decisioning.propositionDismiss": (0.01, -0.2, [("_experience/decisioning/propositionEventType/dismiss", 1)], 0.05),

    
    "directMarketing.emailOpened": (0.2, 0.02, [("directMarketing/opens/value", 1.0)], 24),
    "directMarketing.emailClicked": (0.05, 0.1, [("directMarketing/clicks/value", 1.0)], 0.5),
    "directMarketing.emailSent": (0.5, 0.005, [("directMarketing/sends/value", 1.0)], 0),
    
    "web.formFilledOut": (0.0, 0.0, [("web/webPageDetails/name", "subscriptionForm")], 0),

}

event_dependencies = {
    "advertising.impressions": ["advertising.clicks"],
    "directMarketing.emailSent": ["directMarketing.emailOpened"],
    "directMarketing.emailOpened": ["directMarketing.emailClicked"],
    "directMarketing.emailClicked": ["web.webpagedetails.pageViews"],
    "web.webpagedetails.pageViews": ["web.webinteraction.linkClicks", "commerce.productViews", "decisioning.propositionDisplay"],
    "commerce.productViews": ["commerce.purchases"],
    "decisioning.propositionDisplay": ["decisioning.propositionInteract", "decisioning.propositionDismiss"]
    
}

Define utility functions that will be used to implement the event simulation

In [117]:
import numpy as np
from datetime import datetime

def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

In [118]:

def create_data_for_n_users(n_users, first_user):
  
  N_USERS = n_users
  FIRST_USER = first_user
  
  N_WEEKS = 10
  GLOBAL_START_DATE = datetime.now() - timedelta(weeks=12)
  GLOBAL_END_DATE = GLOBAL_START_DATE + timedelta(weeks=N_WEEKS)

  events = []

  for user in range(N_USERS):
        user_id = FIRST_USER + user
        user_events = []
        base_events = {}
        for event_type in ["advertising.impressions", "web.webpagedetails.pageViews", "directMarketing.emailSent"]:
            n_events = np.random.poisson(advertising_events[event_type][0] * N_WEEKS)
            times = []
            for _ in range(n_events):
                #times.append(random_date(GLOBAL_START_DATE, GLOBAL_END_DATE)
                times.append(random_date(GLOBAL_START_DATE, GLOBAL_END_DATE).isoformat())

            base_events[event_type] = times

        for event_type, dependent_event_types in event_dependencies.items():

            if event_type in base_events:
                #for each originating event
                for event_time in base_events[event_type]:
                    #Look for possible later on events
                    for dependent_event in dependent_event_types:
                                n_events = np.random.poisson(advertising_events[dependent_event][0] * N_WEEKS)
                                times = []
                                for _ in range(n_events):
                                    #times.append(event_time + timedelta(hours = np.random.exponential(advertising_events[dependent_event][3])))
                                    new_time = datetime.fromisoformat(event_time) + timedelta(hours = np.random.exponential(advertising_events[dependent_event][3]))
                                    times.append(new_time.isoformat())
                                base_events[dependent_event] = times


        for event_type, times in base_events.items():
            for time in times:
                user_events.append({"userId": user_id, "eventType": event_type, "timestamp": time})

        user_events = sorted(user_events, key = lambda x: (x["userId"], x["timestamp"]))


        cumulative_probability = 0.001
        subscribed = False
        for event in user_events:
            cumulative_probability = min(1.0, max(cumulative_probability + advertising_events[event["eventType"]][1], 0))
            event["subscriptionPropensity"] = cumulative_probability
            if subscribed == False and "directMarketing" not in event["eventType"] and "advertising" not in event["eventType"]:
                subscribed = np.random.binomial(1, cumulative_probability) > 0
                if subscribed:
                    subscriptiontime = (datetime.fromisoformat(event["timestamp"]) + timedelta(seconds = 60)).isoformat()
                    #subscriptiontime = event["timestamp"] + timedelta(seconds = 60)
                    user_events.append({"userId": user_id, "eventType": "web.formFilledOut",  "timestamp": subscriptiontime})
            event["subscribed"] = subscribed
        user_events = sorted(user_events, key = lambda x: (x["userId"], x["timestamp"]))

        events = events + user_events
  return events

In [119]:
def normalize_ecid(ecid_part):
    ecid_part_str = str(abs(ecid_part))
    if len(ecid_part_str) != 19:
        ecid_part_str = "".join([str(x) for x in range(
            0, 19 - len(ecid_part_str))]) + ecid_part_str
    return ecid_part_str

In [120]:

def get_ecid(email):
    """
    The ECID must be two valid 19 digit longs concatenated together
    """
    ecidpart1, ecidpart2 = mmh3.hash64(email)
    ecid1, ecid2 = (normalize_ecid(ecidpart1), normalize_ecid(ecidpart2))
    return ecid1 + ecid2

In [127]:
# Define the data that goes into an email event payload
def create_email_event(user_id, event_type, timestamp):
  """
  Combines previous methods to create various type of email events
  """
  
  if event_type == "directMarketing.emailSent":
    directMarketing = {"emailDelivered": {"value": 1.0}, 
                       "sends": {"value": 1.0}, 
                       "emailVisitorID": user_id,
                       "hashedEmail": ''.join(random.choices(string.ascii_letters + string.digits, k=10)),
                       "messageID": str(uuid.uuid4()),
                      }
  elif event_type == "directMarketing.emailOpened":
    directMarketing = {"offerOpens": {"value": 1.0}, 
                     "opens": {"value": 1.0}, 
                     "emailVisitorID": user_id,
                     "messageID": str(uuid.uuid4()),
                    }
  elif event_type == "directMarketing.emailClicked":
    directMarketing = {"clicks": {"value": 1.0}, 
                     "offerOpens": {"value": 1.0}, 
                     "emailVisitorID": user_id,
                     "messageID": str(uuid.uuid4()),
                    }
  return {
    "directMarketing": directMarketing,
    "web": None,
    "_id": str(uuid.uuid4()),
    "eventMergeId": None,
    "eventType": event_type,
    f"_{tenant_id}": {"userId":get_ecid(user_id)},
    "producedBy": "databricks-synthetic",
    "timestamp": timestamp
  }

In [128]:
# Define the data that goes into a web event payload 
def create_web_event(user_id, event_type, timestamp):
  """
  Combines previous methods to creat various type of web events
  """
  url = f"http://www.{''.join(random.choices(string.ascii_letters + string.digits, k=5))}.com"
  ref_url = f"http://www.{''.join(random.choices(string.ascii_letters + string.digits, k=5))}.com"
  name = ''.join(random.choices(string.ascii_letters + string.digits, k=5))
  isHomePage = random.choice([True, False])
  server = ''.join(random.choices(string.ascii_letters + string.digits, k=10))
  site_section = ''.join(random.choices(string.ascii_letters, k=2))
  view_name = ''.join(random.choices(string.ascii_letters, k=3))
  region = ''.join(random.choices(string.ascii_letters + string.digits, k=5))
  interaction_type = random.choice(["download", "exit", "other"])
  web_referrer = random.choice(["internal", "external", "search_engine", "email", "social", "unknown", "usenet", "typed_bookmarked"])
  base_web = {"webInteraction": {"linkClicks": {"value": 0.0}, 
                                 "URL": url, 
                                 "name": name,
                                "region": region,
                                "type": interaction_type},
              "webPageDetails": {"pageViews": {"value": 1.0},
                                 "URL": url,
                                 "isErrorPage": False,
                                 #"isHomepage": isHomePage,
                                 "name": name,
                                 "server": server,
                                 "siteSection": site_section,
                                 "viewName": view_name
                                },
              "webReferrer": {
                "URL": ref_url,
                "type": web_referrer
              }
             }
  if event_type in ["advertising.clicks", "commerce.purchases", "web.webinteraction.linkClicks", "web.formFilledOut", 
                   "decisioning.propositionInteract", "decisioning.propositionDismiss"]:
    base_web["webInteraction"]["linkClicks"]["value"] = 1.0

  return {
    "directMarketing": None,
    "web": base_web,
    "_id": str(uuid.uuid4()),
    "eventMergeId": None,
    "eventType": event_type,
    f"_{tenant_id}": {"userId":get_ecid(user_id)},
    "producedBy": "databricks-synthetic",
    "timestamp": timestamp
  }

In [123]:
    
def create_xdm_event(user_id, event_type, timestamp):
  """
  The final 'event factory' method that converts an event into an XDM event
  """
  if "directMarketing" in event_type:
    return create_email_event(user_id, event_type, timestamp)
  else: 
    return create_web_event(user_id, event_type, timestamp)

# 3. Ingest sythetic data into AEP dataset

We'll now use the functions defined above to simulate sequences of Experience Events for a number of users, then ingest the simulated event data into the Experience Event dataset we create above.

For each batch, we will:
1. Initialize a batch to ingest to our Experience Event dataset
2. Generate a sequence of simulate events using the `create_data_for_n_users` function
3. Format the events into XDM Experience Event payloads using the `create_xdm_event` function
4. Add the synthetic data to the batch
5. Close the batch

First create a connection to the AEP batch ingestion API:

In [124]:
from aepp import ingestion
ingest_conn = ingestion.DataIngestion()

In [ ]:
def ingestSyntheticBatch(
        ingest_conn: ingestion.DataIngestion,
        n_users: int = 10000,
        first_user_id: int = 1,
        event_dataset: str = None,
        profile_dataset: str = None) -> str:
    if not events and not profiles:
        raise AttributeError('At least one of "events" or "profiles" must be True')
    if event_dataset:
        # Generate batch of synthetic events data

        # Initialize batch creation

        # Upload data

        # Complete the batch

Then repeat the sequences of actions described above to generate and ingest simulated events for the desired number of batches.

In [129]:
num_batches = 10
batch_size = 10000
dataset_id = dataset_ee_id

batch_ids = []
for batch_index in range(num_batches):
  first_user_id_for_batch = batch_index * batch_size
  batch_events = create_data_for_n_users(batch_size, first_user_id_for_batch)
  batch_data = [create_xdm_event(f"synthetic-user-{x['userId']}@emailsim.io", x["eventType"], x["timestamp"]) for x in batch_events]
  
  # Initialize batch creation
  batch_res = ingest_conn.createBatch(
    datasetId = dataset_id
  )
  #print(str(batch_res))
  batch_id = batch_res["id"]
  print(f"Processing batch {batch_index + 1}/{num_batches} with ID {batch_id}")
  
  # Upload XDM data
  file_path = f"batch-synthetic-{batch_id}"
  ingest_conn.uploadSmallFile(
    batchId = batch_id,
    datasetId = dataset_id,
    filePath = batch_id,
    data = batch_data
  )
  
  # Complete the batch
  ingest_conn.uploadSmallFileFinish(
    batchId = batch_id
  )
  
  # Store the batch ID to check status
  batch_ids.append(batch_id)

Processing batch 1/10 with ID 01H9K8K64GDPDP4XEFA7AFQVS1
Processing batch 2/10 with ID 01H9K8KSRQPNBDBGDR9W5DYGZB
Processing batch 3/10 with ID 01H9K8MQ6JVW1FJFG407C47V2S
Processing batch 4/10 with ID 01H9K8NE5Z12K48FNW8YAEC2QM
Processing batch 5/10 with ID 01H9K8P1030D5TK7GRRFC0H4CA
Processing batch 6/10 with ID 01H9K8PMEJ8763QN5J8AFQVZG5
Processing batch 7/10 with ID 01H9K8QE71R9G17KYZTF482MFM
Processing batch 8/10 with ID 01H9K8R84H67G0W13ME0W9BK0M
Processing batch 9/10 with ID 01H9K8RWWRY19VXEAPSAHHFWP0
Processing batch 10/10 with ID 01H9K8SFCASX2QRC2JE08PYEEG


**Note**: Batches are ingested asynchronously in AEP. It may take some time for all the data generated here to be available in your dataset depending on how your AEP organization has been provisioned. You can check ingestion status for all your batches in [the dataset page of your AEP UI](https://experience.adobe.com/#/@TENANT/sname:SANDBOX/platform/dataset/browse/DATASETID)

You can also check the ingestion status from the notebook by running the following cell:

In [131]:
from aepp import catalog
import time
cat_conn = catalog.Catalog()

all_ingested = False
while not all_ingested:
  incomplete_batches = cat_conn.getBatches(
    limit=min(100, num_batches),
    n_results=num_batches,
    output="dataframe",
    dataSet=dataset_id,
    status="staging"
  )
  
  num_incomplete_batches = len(incomplete_batches)
  if num_incomplete_batches == 0:
    print("All batches have been ingested")
    all_ingested = True
  else:
    print(f"Remaining batches being ingested: {num_incomplete_batches}")
    time.sleep(30)

Remaining batches being ingested: 1


KeyboardInterrupt: 